In [2]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

tf.random.set_seed(777)

2.3.0
1.18.5


In [4]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#data를 list로 선언 후 array로 바꿔준다
x_data = np.asarray(x_data, dtype = np.float32)
y_data = np.asarray(y_data, dtype = np.float32)

In [9]:
#class의 개수
#3가지 중 하나로 구분한다
nb_classes = 3

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [12]:
#Weight, bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name = 'weight')
b = tf.Variable(tf.random.normal((nb_classes, )), name = 'bias')

variables = [W, b]

print(W, b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 1.7266569 ,  0.93108284, -0.43655586],
       [-1.2439743 , -0.45300987, -0.51410425],
       [ 1.2241375 ,  2.1267812 , -1.6276842 ],
       [-0.11408016, -0.08377677,  0.12709002]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-0.27697366, -2.3162541 ,  1.486129  ], dtype=float32)>


In [14]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[5.1594371e-01 3.7466004e-01 1.0939622e-01]
 [5.1771075e-01 4.8226342e-01 2.5805661e-05]
 [6.9122940e-01 3.0876419e-01 6.4158853e-06]
 [4.4173509e-01 5.5826491e-01 1.9998740e-09]
 [6.3159358e-04 9.9936825e-01 1.5016340e-07]
 [3.1004906e-02 9.6899486e-01 2.4400759e-07]
 [5.4803921e-04 9.9945194e-01 4.6149586e-09]
 [9.7796321e-05 9.9990225e-01 1.2556407e-10]], shape=(8, 3), dtype=float32)


In [15]:
#softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[9.9697566e-01 3.0241790e-03 1.1561164e-07]], shape=(1, 3), dtype=float32)


In [16]:
#softmax cost function = -y * log(hypo) ??
#강의 다시 듣기
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis = 1)
    cost_mean = tf.reduce_mean(cost)
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(5.2614083, shape=(), dtype=float32)


In [17]:
# ??
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [18]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.42803454,  0.30828205, -0.73631656],
       [-1.2808793 ,  1.7535259 , -0.47264674],
       [-0.8108007 ,  1.6721138 , -0.86131316],
       [-0.78523517,  1.6465507 , -0.86131555]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.02486265,  0.33645874, -0.3613214 ], dtype=float32)>]


In [20]:
#학습 시키기
def fit(X, Y, epochs = 2000, verbose = 100):
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1)
    
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i == 0) | ((i + 1) % verbose == 0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))

fit(x_data, y_data)

Loss at epoch 1: 4.102696
Loss at epoch 100: 0.646512
Loss at epoch 200: 0.568315
Loss at epoch 300: 0.515267
Loss at epoch 400: 0.468391
Loss at epoch 500: 0.423284
Loss at epoch 600: 0.378449
Loss at epoch 700: 0.333312
Loss at epoch 800: 0.287950
Loss at epoch 900: 0.244896
Loss at epoch 1000: 0.220589
Loss at epoch 1100: 0.210063
Loss at epoch 1200: 0.200549
Loss at epoch 1300: 0.191842
Loss at epoch 1400: 0.183843
Loss at epoch 1500: 0.176471
Loss at epoch 1600: 0.169653
Loss at epoch 1700: 0.163332
Loss at epoch 1800: 0.157454
Loss at epoch 1900: 0.151976
Loss at epoch 2000: 0.146857


In [21]:
#prediction check

sample_data = [[2, 1, 3, 2]] #answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype = np.float32)

a = hypothesis(sample_data)
print (a)
print(tf.argmax(a, 1))

tf.Tensor([[0.00135109 0.06746072 0.9311882 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [22]:
b = hypothesis(x_data)
print(b)

tf.Tensor(
[[3.9563643e-06 1.3164548e-03 9.9867958e-01]
 [1.3510946e-03 6.7460753e-02 9.3118823e-01]
 [2.9748232e-08 1.4705136e-01 8.5294861e-01]
 [7.3428276e-07 8.6910748e-01 1.3089177e-01]
 [2.5018707e-01 7.3944026e-01 1.0372761e-02]
 [1.3216078e-01 8.6721033e-01 6.2882266e-04]
 [7.5919098e-01 2.4070956e-01 9.9491197e-05]
 [9.2029667e-01 7.9701401e-02 1.9764948e-06]], shape=(8, 3), dtype=float32)


In [23]:
print(tf.argmax(b, 1))

tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [24]:
print(tf.argmax(y_data, 1))

tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [25]:
#Convert as Class

class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)),name = 'weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name = 'bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis = 1))
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)
            return grads
    
    def fit(self, X, Y, epochs = 2000, verbose = 500):
        optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1)
        
        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i == 0) | ((i + 1) % verbose == 0):
                print('Loss as epoch %d: %f' %(i + 1, self.cost_fn(X,Y).numpy()))

model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

Loss as epoch 1: 7.899120
Loss as epoch 500: 0.493022
Loss as epoch 1000: 0.268561
Loss as epoch 1500: 0.194537
Loss as epoch 2000: 0.159601
